In [6]:
import numpy as np
from astropy import constants as const
from astropy import units as u

c = const.c.value
G = const.G.value

In [ ]:
l = 1

m_1 = 1
m_2 = 1

def μ(m_1, m_2):
    μ = m_1*m_2/(m_1 + m_2)
    return μ

def U_eff(l, μ, r):
    return l**2/(2*μ*r**2) - G*m_1*m_2/r
 
def findv0(E, l, μ, r):
    v_0 = np.abs(np.sqrt(2*(E - U_eff(l, μ, r))/μ))
    return v_0



$$U_\mathrm{eff}(r) = \frac 12 \frac{l^2}{\mu r^2} + U(r) \qquad \text{and} \qquad U(r) = \frac k r$$

$$E_{i, \mathrm{eff, CM}} (\dot {\mathbf r}_i^2) = \frac 1 2 \mu \dot {\mathbf r}_i + U_\mathrm{eff}(\mathbf r_i)$$

